In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
# from pyspark.sql import types
# from pyspark.sql.functions import udf as F

In [2]:
# On terminal ../data
! ls ../data
# !gsutil -m cp -r part/ gs://data_lake_nyc-taxi-359621/part 
#         👆 -m for multithreaded upload

part   raw    report


In [3]:
# spark = (
#     SparkSession
#     .builder
#     .master("local[4]")
#     .appName("pyspark-etl.sandbox")
#     .getOrCreate()
# )

In [4]:
CREDS_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
SPARK_HOME = os.getenv("SPARK_HOME")

# for path in (CREDS_PATH, SPARK_HOME):
#     print(path)

In [5]:
conf = (
    SparkConf()
    .setMaster("local[4]")
    .setAppName("pyspark-local-master-gcs.sandbox")
    .set("spark.jars", f"{SPARK_HOME}/jars/gcs-connector-hadoop3-latest.jar")
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", CREDS_PATH)
)

In [6]:
sc = SparkContext(conf = conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", CREDS_PATH)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

22/09/02 10:22:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
spark = (
    SparkSession.builder
    .config(conf = sc.getConf())
    .getOrCreate()
    )

In [8]:
df_green = spark.read.parquet("gs://data_lake_nyc-taxi-359621/part/fhv/2021/*")

In [9]:
df_green.count()

3846328

In [10]:
spark.stop()